#### Preprocess

In [1]:
import pandas as pd, numpy as np
from tqdm import tqdm

In [2]:
df_train = pd.read_csv("bbdc_2023_AWI_data_develop_professional.csv")

In [3]:
nan_counts = df_train.applymap(lambda x: x.count('NA'))

In [4]:
clean_cols = df_train.columns[0].split(';')

In [5]:
clean_values = list(map(lambda x: x[0].split(';'), df_train.values))

In [6]:
df_train_clean = pd.DataFrame(clean_values[1:], columns=clean_cols)

In [7]:
df_train_clean["Datum"] = pd.to_datetime(df_train_clean["Datum"], yearfirst=True, dayfirst=True)

In [8]:
def handle_time(df):
    c=0
    values = []
    for row in tqdm(df['Uhrzeit'], total=len(df)):
        try:
            pd.to_datetime(row)
            values.append(row)
        except ValueError:
            df_train_clean.loc[c, 'Uhrzeit'] = "NA"
        c+=1
    df["Uhrzeit"] = df["Uhrzeit"].replace("NA", pd.to_datetime(values).mean())
    df["Uhrzeit"] = pd.to_datetime(df["Uhrzeit"], yearfirst=True, dayfirst=True)
    return df

In [9]:
df_train_clean = handle_time(df_train_clean)

100%|██████████| 17531/17531 [00:03<00:00, 5178.98it/s]
/tmp/ipykernel_98938/3943007440.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Uhrzeit"] = df["Uhrzeit"].replace("NA", pd.to_datetime(values).mean())


In [10]:
min_date = df_train_clean['Datum'].min()

In [11]:
df_train_clean["Datum_Delta"] = (df_train_clean['Datum'] - min_date)  / np.timedelta64(1,'Y')

In [12]:
min_time = df_train_clean['Uhrzeit'].min()

In [13]:
df_train_clean["Uhrzeit_Delta"] = (df_train_clean['Uhrzeit'] - min_time)  / np.timedelta64(1,'h')

In [14]:
df_train_clean['Datum_Sin'] = np.sin(df_train_clean["Datum_Delta"] * (2 * np.pi))
df_train_clean['Datum_Cos'] = np.cos(df_train_clean["Datum_Delta"] * (2 * np.pi))

In [15]:
def handle_missing_numeric(df, col):
    values = []
    for c, row in enumerate(df[col]):
        if row != "NA":
            values.append(float(row.strip("?")))
        try:
            float(row)
        except ValueError:
            df.loc[c, col] = str(row).strip("?")
    df[col] = df[col].replace("NA", str(np.mean(values))).astype(float)
    return df

In [16]:
for col in tqdm(['SECCI', 'Temperatur', 'Salinität', 'NH4', 'NOx', 'NO2', 'NO3', 'PO4', 'SiO4']):
    df_train_clean = handle_missing_numeric(df_train_clean, col)

100%|██████████| 9/9 [00:06<00:00,  1.41it/s]


In [17]:
df_train_clean.drop(['Datum', 'Uhrzeit'], axis=1, inplace=True)

In [18]:
cols = df_train_clean.columns
cols = list(cols[-4:])+list(cols[0:-4])

In [19]:
df_train_clean = df_train_clean[cols]

In [20]:
df_train_clean.to_csv('data_train_additional.csv', index=False)

### Start

In [21]:
import pandas as pd, numpy as np
from tqdm import tqdm

In [22]:
df_train_clean = pd.read_csv('data_train_additional.csv')

### Test

In [23]:
df_test = pd.read_csv("bbdc_2023_AWI_data_evaluate_skeleton_professional.csv")

In [24]:
clean_cols = df_test.columns[0].split(';')

In [25]:
test_values = list(map(lambda x: x[0].split(';'), df_test.values))

In [26]:
df_test_clean = pd.DataFrame(test_values[1:], columns=clean_cols)

In [27]:
df_test_clean["Datum"] = pd.to_datetime(df_test_clean["Datum"], yearfirst=True, dayfirst=True)

In [28]:
df_test_clean = handle_time(df_test_clean)

100%|██████████| 1365/1365 [00:00<00:00, 5222.68it/s]
/tmp/ipykernel_98938/3943007440.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Uhrzeit"] = df["Uhrzeit"].replace("NA", pd.to_datetime(values).mean())
/tmp/ipykernel_98938/3943007440.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Uhrzeit"] = pd.to_datetime(df["Uhrzeit"], yearfirst=True, dayfirst=True)


In [29]:
df_test_clean["Datum_Delta"] = (df_test_clean['Datum'] - min_date)  / np.timedelta64(1,'Y')

In [30]:
df_test_clean["Uhrzeit_Delta"] = (df_test_clean['Uhrzeit'] - min_time)  / np.timedelta64(1,'h')

In [31]:
df_test_clean['Datum_Sin'] = np.sin(df_test_clean["Datum_Delta"] * (2 * np.pi))
df_test_clean['Datum_Cos'] = np.cos(df_test_clean["Datum_Delta"] * (2 * np.pi))

In [32]:
cols = df_test_clean.columns
cols = list(cols[-4:])+list(cols[0:-4])
df_test_clean = df_test_clean[cols]

In [33]:
df_test_clean.drop(['Datum', 'Uhrzeit'], axis=1, inplace=True)

In [34]:
df_test_clean.to_csv("data_test_additional.csv", index=False)